## <center>Ensemble models from machine learning: an example of wave runup and coastal dune erosion</center>
### <center>Tomas Beuzen<sup>1</sup>, Evan B. Goldstein<sup>2</sup>, Kristen D. Splinter<sup>1</sup></center>
<center><sup>1</sup>Water Research Laboratory, School of Civil and Environmental Engineering, UNSW Sydney, NSW, Australia</center>

<center><sup>2</sup>Department of Geography, Environment, and Sustainability, University of North Carolina at Greensboro, Greensboro, NC, USA</center>


This notebook contains the code required to reproduce the analysis and figures presented in the manuscript "*Ensemble models from machine learning: an example of wave runup and coastal dune erosion*" by Beuzen et al.

**Citation:** Beuzen, T, Goldstein, E.B., Splinter, K.S. (In Review). Ensemble models from machine learning: an example of wave runup and coastal dune erosion, Natural Hazards and Earth Systems Science, SI Advances in computational modeling of geoprocesses and geohazards.

### Table of Contents:
* [Imports](#bullet-0)
* [Figure 3](#bullet-1)
* [Figure 4](#bullet-2)
* [Figure 5](#bullet-3)
* [Figure 7](#bullet-4)
* [Figure 8](#bullet-5)
* [Figure 9](#bullet-6)

## Imports <a class="anchor" id="bullet-0"></a>

In [ ]:
# Required imports
# Standard computing packages
import pickle, gzip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Gaussian Process tools
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

# Custom functions
from support_functions import train_test_mda, LEH04ensembles

# Notebook functionality
%matplotlib inline
%load_ext autoreload

## Figure 3 <a class="anchor" id="bullet-1"></a>
In this section, we will load and visualise the wave, beach slope, and runup data we will use to develop the Gaussian process (GP) runup predictor.

In [ ]:
# Read in .csv data file as a pandas dataframe
df = pd.read_csv('../data_repo_temporary/lidar_runup_data.csv',parse_dates=True,index_col=0,dayfirst=True).dropna()
# Print the size and head of the dataframe
print('Data size:', df.shape)
df.head()

In [ ]:
# This cell plots Figure 3, showing histograms of the data
# Initialize the figure and axes
fig, axes = plt.subplots(2,2,figsize=(6,6))
plt.tight_layout(w_pad=0.1, h_pad=3)
# Subplot (0,0): Hs
ax = axes[0,0]
ax.hist(df.Hs,37,color=(0.6,0.6,0.6),edgecolor='k',lw=0.5) # Plot histogram
ax.set_xlabel('H$_s$ (m)',fontweight='bold') # Format plot
ax.set_ylabel('Frequency',fontweight='bold')
ax.set_xticks((0,1.5,3,4.5))
ax.set_xlim((0,4.5))
ax.set_ylim((0,1200))
ax.grid(lw=0.5,alpha=0.7)
ax.text(-1.1, 1250, 'A)', fontweight='bold', fontsize=12)
ax.tick_params(direction='in')
ax.set_axisbelow(True)
# Subplot (0,1): Tp
ax = axes[0,1]
ax.hist(df.Tp,20,color=(0.6,0.6,0.6),edgecolor='k',lw=0.5) # Plot histogram
ax.set_xlabel('T$_p$ (s)',fontweight='bold') # Format plot
ax.set_xticks((0,6,12,18))
ax.set_xlim((0,18))
ax.set_ylim((0,1200))
ax.set_yticklabels([])
ax.grid(lw=0.5,alpha=0.7)
ax.text(-2.1, 1250, 'B)', fontweight='bold', fontsize=12)
ax.tick_params(direction='in')
ax.set_axisbelow(True)
# Subplot (1,0): beta
ax = axes[1,0]
ax.hist(df.beach_slope,20,color=(0.6,0.6,0.6),edgecolor='k',lw=0.5) # Plot histogram
ax.set_xlabel(r'$\beta$',fontweight='bold') # Format plot
ax.set_ylabel('Frequency',fontweight='bold')
ax.set_xticks((0,0.1,0.2,0.3))
ax.set_xlim((0,0.3))
ax.set_ylim((0,1200))
ax.grid(lw=0.5,alpha=0.7)
ax.text(-0.073, 1250, 'C)', fontweight='bold', fontsize=12)
ax.tick_params(direction='in')
ax.set_axisbelow(True)
# Subplot (1,1): R2
ax = axes[1,1]
ax.hist(df.runup,25,color=(0.9,0.2,0.2),edgecolor='k',lw=0.5) # Plot histogram
ax.set_xlabel('R$_2$ (m)') # Format plot
ax.set_xticks((0,1,2,3))
ax.set_xlim((0,3))
ax.set_ylim((0,1200))
ax.set_yticklabels([])
ax.grid(lw=0.5,alpha=0.7)
ax.text(-0.35, 1250, 'D)', fontweight='bold', fontsize=12)
ax.tick_params(direction='in')
ax.set_axisbelow(True);

## Figure 4 <a class="anchor" id="bullet-2"></a>
In this section we will first split our data into training, validation and testing sets, before developing and evaluating the performance of the GP predictor.


The training dataset is selected using a Maximum Dissimilarity Algorithm (MDA). The validation and testing sets are then selected by equally, randomly splitting all the data not selected as part of the trianing dataset. Extensive preliminary testing showed that a training dataset size of 5% was optimum for maximizing GP performance and minimizing computational complexity. See **Section 3.2** of the manuscript for further discussion.

We then standardize the data for use in the GP by removing the mean and scaling to unit variance. This does not really affect GP performance but improves computational efficiency (see sklearn documentation for more information). Note that the scaling regime is only based on the training dataset, this scaling is then applied to the validation and testing datasets separately.

A kenerl must be specified to develop the GP. Many kernels were trialled in initial GP development. The final kernel is a combination of the RBF and WhiteKernel. See **Section 2.1** and **Section 2.2** of the manuscript for further discussion.

In [ ]:
# Split data into training/testing sets
X = df.drop(columns=df.columns[-1]) # Drop the last column to retain input features
y = df[[df.columns[-1]]]            # The last column is the predictand
# Select the training dataset using Maximum Dissimilarity Algorithm (MDA).
X_train, X_temp, y_train, y_temp = train_test_mda(X,
                                                  y,
                                                  sample_size=0.05,
                                                  dist_measure='euclidean',
                                                  standardize = True)
# Now, equally split remaining data into validation and test datasets
X_validation, X_test, y_validation, y_test = train_test_split(X_temp,
                                                              y_temp,
                                                              test_size=0.5,
                                                              shuffle=False,
                                                              random_state=123)
# Print dataset characteristics
print('\033[4mDATASETS\033[0m')
print('Training Size:   ' + str(len(X_train)))
print('Validation Size: ' + str(len(X_validation)))
print('Testing Size:    ' + str(len(X_test)))

In [ ]:
# Standardize data for use in the GP
scaler = StandardScaler()
scaler.fit(X_train) # Fit the scaler to the training data
X_train_scaled = scaler.transform(X_train) # Scale training data
X_validation_scaled = scaler.transform(X_validation) # Scale validation data
X_test_scaled = scaler.transform(X_test) # Scale testing data

In [ ]:
# Specify the kernel to use in the GP
kernel = RBF(0.1, (1e-2, 1e2)) + WhiteKernel(1,(1e-2,1e2))

In [ ]:
# Train GP model on training dataset
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=True, random_state=123)
gp.fit(X_train_scaled, y_train)
# Evaluate performance on the validation dataset
y_validation_predictions, sigma_validation = gp.predict(X_validation_scaled, return_std=True)
print('GP RMSE on validation data =', format(np.sqrt(mean_squared_error(y_validation,y_validation_predictions)),'.2f'), 'm')
# Evaluate performance on the testing dataset
y_test_predictions, sigma_test = gp.predict(X_test_scaled, return_std=True)
print('GP RMSE on test data =', format(np.sqrt(mean_squared_error(y_test,y_test_predictions)),'.2f'), 'm')

In [ ]:
# This cell plots Figure 4, comparing GP predictions to observations for the testing dataset
# Initialize the figure and axes
fig, axes = plt.subplots(figsize=(6,6))
plt.tight_layout(pad=2.2)
# Plot and format
axes.scatter(y_test,y_test_predictions,s=10,c='b',marker='.')
axes.plot([0,4],[0,4],'k--')
axes.set_ylabel('Predicted R$_2$ (m)')
axes.set_xlabel('Observed R$_2$ (m)')
axes.grid(lw=0.5,alpha=0.7)
axes.set_xlim(0,2.5)
axes.set_ylim(0,2.5)
# Print some statistics
print('GP RMSE on test data =', format(np.sqrt(mean_squared_error(y_test,y_test_predictions)),'.2f'))
print('GP bias on test data =', format(np.mean(y_test_predictions-y_test.values),'.2f'))
print('correlation-squared =', format(pearsonr(y_test_predictions,y_test.values)[0][0]**2,'.2f'))

## Figure 5 <a class="anchor" id="bullet-3"></a>
This section explores how we can draw random samples from the GP to explain scatter in the runup predictions. We randomly draw 100 samples from the GP and calculate how much of the scatter in the runup predictions is captured by the ensemble envelope for different ensemble sizes. The process is repeated 100 times for robustness. See **Section 3.3** of the manuscript for further discussion.

In [ ]:
# Draw 100 samples from the GP model using the testing dataset
GP_draws = gp.sample_y(X_test_scaled, n_samples=100, random_state=123).squeeze() # Draw 100 random samples from the GP
# Initialize result arrays
perc_ens = np.zeros((100,100)) # Initialize ensemble capture array
perc_err = np.zeros((100,))   # Initialise arbitray error array
# Loop to get results
for i in range(0,perc_ens.shape[0]):
    # Caclulate capture % in envelope created by adding arbitrary, uniform error to mean GP prediction
    lower = y_test_predictions*(1-i/100) # Lower bound
    upper = y_test_predictions*(1+i/100) # Upper bound
    perc_err[i] = sum((np.squeeze(y_test)>=np.squeeze(lower)) & (np.squeeze(y_test)<=np.squeeze(upper)))/y_test.shape[0] # Store percent capture
    for j in range(0,perc_ens.shape[1]):
        ind = np.random.randint(0,perc_ens.shape[0],i+1) # Determine i random integers
        lower = np.min(GP_draws[:,ind],axis=1) # Lower bound of ensemble of i random members
        upper = np.max(GP_draws[:,ind],axis=1) # Upper bound of ensemble of i random members
        perc_ens[i,j] = sum((np.squeeze(y_test)>=lower) & (np.squeeze(y_test)<=upper))/y_test.shape[0] # Store percent capture

In [ ]:
# This cell plots Figure 5, showing how samples from the GP can help to capture uncertainty in predictions
# Initialize the figure and axes
fig, axes = plt.subplots(1,2,figsize=(9,4))
plt.tight_layout()
lim = 0.95 # Desired limit to test
# Plot ensemble results
ax = axes[0]
perc_ens_mean = np.mean(perc_ens,axis=1)
ax.plot(perc_ens_mean*100,'k-',lw=2)
ind = np.argmin(abs(perc_ens_mean-lim)) # Find where the capture rate > lim
ax.plot([ind,ind],[0,perc_ens_mean[ind]*100],'r--')
ax.plot([0,ind],[perc_ens_mean[ind]*100,perc_ens_mean[ind]*100],'r--')
ax.set_xlabel('# Draws from GP')
ax.set_ylabel('Observations captured \n within ensemble range (%)')
ax.grid(lw=0.5,alpha=0.7)
ax.minorticks_on()
ax.set_xlim(0,100);
ax.set_ylim(0,100);
ax.text(-11.5, 107, 'A)', fontweight='bold', fontsize=12)
print('# draws needed for ' + format(lim*100,'.0f') + '% capture = ' + str(ind))
print('Mean/Min/Max for ' + str(ind) + ' draws = '
      + format(np.mean(perc_ens[ind,:])*100,'.1f') + '%/'
      + format(np.min(perc_ens[ind,:])*100,'.1f') + '%/'
      + format(np.max(perc_ens[ind,:])*100,'.1f') + '%')
# Plot arbitrary error results
ax = axes[1]
ax.plot(perc_err*100,'k-',lw=2)
ind = np.argmin(abs(perc_err-lim))  # Find where the capture rate > lim
ax.plot([ind,ind],[0,perc_err[ind]*100],'r--')
ax.plot([0,ind],[perc_err[ind]*100,perc_err[ind]*100],'r--')
ax.set_xlabel('% Error added to mean GP estimate')
ax.grid(lw=0.5,alpha=0.7)
ax.minorticks_on()
ax.set_xlim(0,100);
ax.set_ylim(0,100);
ax.text(-11.5, 107, 'B)', fontweight='bold', fontsize=12)
print('% added error needed for ' + format(lim*100,'.0f') + '% capture = ' + str(ind) + '%')

## Figure 7 <a class="anchor" id="bullet-4"></a>
We now show how the GP predictor can be used within a dune erosion model to form ensemble predictions of dune erosion. We will import data of 40 profiles from a storm event that impacted Narrabeen Beach, Sydney, Australia in June 2011.  See **Section 4.2** of the manuscript for further information on this dataset. We use the model of Larson et al (2004) (LEH04) with the GP runup predictor to form model ensembles as discussed in **Section 4.1** of the manuscript.


10,000 runup models are drawn from the GP runup predictor and used within the LEH04 model to create an ensemble of dune erosion predictions. This section shows one such prediction for a single profile.

In [ ]:
# Read in beach slope, wave and water level data for the 40 profiles impacted by the June 2011 storm and make predictions
# of runup using the GP
draws = 10000 # We will draw 10,000 ensembles
june2011_waves = pd.read_csv('../data_repo_temporary/june2011_slope_wave_waterlevel_data.csv',index_col=0) # Read the data
water_level = june2011_waves.loc[i+1]['water_level'] # Isolate the water level to be added to R2 predictions
# Initialize result arrays
GP_mean = np.zeros((40,120))
GP_sigma = np.zeros((40,120))
GP_samples = np.zeros((40,120,draws))
# Loop to predict wave runup at each profile
for i in range(40):
    X = scaler.transform(june2011_waves.loc[i+1].drop(columns='water_level'))
    y, sigma = gp.predict(X, return_std=True) # Predict R2
    GP_mean[i,:] = np.squeeze(y) + water_level # add the water level to R2 predictions
    GP_sigma[i,:] = sigma
    y_samples = gp.sample_y(X, n_samples=draws).squeeze() + water_level[:, None]  # sample draws from the GP R2 predictor
    GP_samples[i,:,:] = y_samples

In [ ]:
# Load the morphological data for the 40 profiles, including dune toe position, dune erosion volumes, etc.
with open('../data_repo_temporary/june2011_profile_data.pkl', 'rb') as f:
    june2011_profiles = pickle.load(f) # load the data
# Define some parameters
profile_index = 12; # a profile to analyse
Cs = 0.0015 # Cs value to use in LEH04
# Pull required data out of the june2011_profiles dictionary
profile = data[profile_index] # get the profile data  
dv = profile['dv'] # dune volume
zb = profile['zb'] # dune toe
T = profile['Tp'] # wave period
GP_sample = np.squeeze(GP_samples[profile_index,:,:]) # the GP runup draws for the profile
# Run the LEH04 model
[SigDuneErosionGPD,zbmGPD] = LEH04ensembles(dv,zb,GP_sample,T,Cs)

In [ ]:
# This cell plots Figure 7, showing an example of the GP runup predictor used with the LEH04 model to produce ensemble
# predictions of dune erosion
# Initialize the figure and axes
fig, _ = plt.subplots(figsize=(19/2.54,17/2.54))
plt.tight_layout(pad=0.1,w_pad=0.1,h_pad=0.1)
ax1 = plt.subplot2grid((5, 1), (0, 0), rowspan=3)
ax2 = plt.subplot2grid((5, 1), (3, 0), rowspan=2)
# Define some parameters
t = np.arange(1, 121) # time
ens = 10000 # number of ensemble members to plot
profile['times']=[1,24,47,74,119]; # observed dune characteristics at times through the storm event
profile['zb_12345']=[2.6310,2.8427,2.8849,2.9939,3.0162]
profile['dv_12345']=[0.1301,0.7682,1.2730,1.8308,2.2395]
# Observed Dune Toe Position
ax1.plot(profile['times'],profile['zb_12345'],'.',markersize=15,mfc='pink',mew=1,mec='k',zorder=10,
         label='Observed Dune Toe Position')
# Runup
ax1.plot([],'w',label=' ')
ax1.plot([],'w',label='Runup Predictions:')
ax1.plot(t, np.median(GP_sample[:,:ens],axis=1),'-b',lw=2,label='Ensemble Mean',zorder=10)
ax1.fill_between(t,np.percentile(GP_sample[:,:ens-1],17,axis=1),
                  np.percentile(GP_sample[:,:ens-1],83,axis=1),
                  alpha=0.4,
                  facecolor='royalblue',
                  label='66% Range',zorder=10)
ax1.fill_between(t,np.percentile(GP_sample[:,:ens-1],5,axis=1),
                  np.percentile(GP_sample[:,:ens-1],95,axis=1),
                  alpha=0.3,
                  facecolor='royalblue',
                  label='90% Range',zorder=10)
ax1.fill_between(t,np.percentile(GP_sample[:,:ens-1],0.5,axis=1),
                  np.percentile(GP_sample[:,:ens-1],99.5,axis=1),
                  alpha=0.2,
                  facecolor='royalblue',
                  label='99% Range',zorder=10)
# Modelled Dune Toe Position
ax1.plot([],'w',label=' ')
ax1.plot([],'w',label='Dune Toe Predictions:')
ax1.plot(t, np.mean(zbmGPD[:,:ens],axis=1),'-k',lw=3,label='Ensemble Mean',zorder=1)
ax1.fill_between(t,np.percentile(zbmGPD[:,:ens-1],17,axis=1),
                  np.percentile(zbmGPD[:,:ens-1],83,axis=1),
                  alpha=0.9,
                  facecolor='grey',
                  label='66% Range')
ax1.fill_between(t,np.percentile(zbmGPD[:,:ens-1],5,axis=1),
                  np.percentile(zbmGPD[:,:ens-1],95,axis=1),
                  alpha=0.5,
                  facecolor='grey',
                  label='90% Range')
ax1.fill_between(t,np.percentile(zbmGPD[:,:ens-1],0.5,axis=1),
                  np.percentile(zbmGPD[:,:ens-1],99.5,axis=1),
                  alpha=0.3,
                  facecolor='grey',
                  label='99% Range')
# Formatting
ax1.set_title('Profile 141')
ax1.set_ylabel('Elevation (m)')
ax1.set_yticks(np.linspace(0,4,5))
ax1.set_ylim(0,4)
ax1.set_xlim(0,120)
ax1.set_xticklabels([])
ax1.legend()
handles, labels = ax1.get_legend_handles_labels()
handles = [handles[i] for i in [0,1,2,3,7,8,9,4,5,6,10,11,12]]
labels = [labels[i] for i in [0,1,2,3,7,8,9,4,5,6,10,11,12]]
ax1.legend(handles,labels,loc=6,bbox_to_anchor=(1, 0.5))
ax1.text(-9.4, 3.9, 'A)', fontweight='bold', fontsize=12)
# Observed Dune Erosion
ax2.plot(profile['times'],profile['dv_12345'],'.',markersize=15,mfc='pink',mew=1,mec='k',zorder=10,
         label='Observed Dune Erosion')
# Modelled Dune Toe Position
ax2.plot(t, np.mean(SigDuneErosionGPD[:,:ens],axis=1),'-k',lw=3,label='Ensemble Mean',zorder=9)
ax2.plot([],'w',label=' ')
ax2.plot([],'w',label='Dune Erosion Predictions:    ')
ax2.fill_between(t,np.percentile(SigDuneErosionGPD[:,:ens-1],17,axis=1),
                  np.percentile(SigDuneErosionGPD[:,:ens-1],83,axis=1),
                  alpha=0.9,
                  facecolor='grey',
                  label='66% Range')
ax2.fill_between(t,np.percentile(SigDuneErosionGPD[:,:ens-1],5,axis=1),
                  np.percentile(SigDuneErosionGPD[:,:ens-1],95,axis=1),
                  alpha=0.5,
                  facecolor='grey',
                  label='90% Range')
ax2.fill_between(t,np.percentile(SigDuneErosionGPD[:,:ens-1],0.5,axis=1),
                  np.percentile(SigDuneErosionGPD[:,:ens-1],99.5,axis=1),
                  alpha=0.3,
                  facecolor='grey',
                  label='99% Range')
# Formatting
ax2.set_ylabel('Dune Erosion (m$^3$/m)')
ax2.set_xlabel('Time (hours)')
ax2.set_yticks(np.linspace(0,8,5))
ax2.set_ylim(0,8)
ax2.set_xlim(0,120)
ax2.legend()
handles, labels = ax2.get_legend_handles_labels()
handles = [handles[i] for i in [0,2,3,1,4,5,6]]
labels = [labels[i] for i in [0,2,3,1,4,5,6]]
ax2.legend(handles,labels,loc=6,bbox_to_anchor=(1, 0.5))
ax2.text(-9.4, 7.8, 'B)', fontweight='bold', fontsize=12);

## Figure 8 <a class="anchor" id="bullet-5"></a>
Figure 7 previously showed how the GP predictor + LEH04 model can be used to geenrate ensemble dune erosion predictions. We will now show the performance of the approach on all 40 profiles. Effectively the same code as previously is applied again, but over all 40 profiles and the results are plotted.

In [ ]:
# Define some parameters
Cs = 0.0015 # Cs to use in LEH04
draws = 10000 # # We will use 10,000 ensembles
# Initialize result arrays
obs = np.zeros(len(data))
MeanGP = np.zeros(len(data))
MedianGP = np.zeros(len(data))
MaxGP66 = np.zeros(len(data))
MinGP66 = np.zeros(len(data))
MaxGP90 = np.zeros(len(data))
MinGP90 = np.zeros(len(data))
MaxGP99 = np.zeros(len(data))
MinGP99 = np.zeros(len(data))
errorGPmean = np.zeros(len(data))
errorGPmedian = np.zeros(len(data))
#loop through each profile and make dune erosion predictions
for k in june2011_profiles:
    profile=june2011_profiles[k] # get the profiel data
    obs[k] = june2011_profiles[k]['dv_obs'] # store observed dune erosion           
    # pull all relevant data out of the dictionary
    dv = profile['dv']
    zb = profile['zb']
    T = profile['Tp']
    GP_sample = np.squeeze(GP_samples[k,:,:]) # the GP runup draws for the profile
    # run the LEH04 model
    [SigDuneErosionGPD,zbmGPD] = LEH04ensembles(dv,zb,GP_sample,T,Cs)
    # record the max and min from GP draws for different probabilities
    MeanGP[k] = np.mean(SigDuneErosionGPD[-1,:])
    MedianGP[k] = np.median(SigDuneErosionGPD[-1,:])
    MaxGP66[k] = np.percentile(SigDuneErosionGPD[-1,:],83)
    MinGP66[k] = np.percentile(SigDuneErosionGPD[-1,:],17)
    MaxGP90[k] = np.percentile(SigDuneErosionGPD[-1,:],95)
    MinGP90[k] = np.percentile(SigDuneErosionGPD[-1,:],5)
    MaxGP99[k] = np.percentile(SigDuneErosionGPD[-1,:],99.5)
    MinGP99[k] = np.percentile(SigDuneErosionGPD[-1,:],0.5)
    #GP ensemble mean and median
    errorGPmean[k] = np.absolute(profile['dv_obs'] - MeanGP[k])
    errorGPmedian[k] = np.absolute(profile['dv_obs'] - MedianGP[k])    

In [ ]:
# This cell plots Figure 8, showing predictions for all 40 profiles
# Initialize the figure and axes
fig, axes = plt.subplots(1,1,figsize=(10,6))
# Plot
x = np.arange(1, len(obs)+1)
axes.plot(x,obs,'o',linestyle='-',color='pink',mfc='pink',mew=1,mec='k',zorder=10,label='Observed')
axes.plot(x,MeanGP,'k',marker='o',label='Ensemble Mean')

axes.fill_between(x,
                  MinGP66,
                  MaxGP66,
                  alpha=0.9,
                  facecolor='grey',
                  label='66% Range')
axes.fill_between(x,
                  MinGP90,
                  MaxGP90,
                  alpha=0.5,
                  facecolor='grey',
                  label='90% Range')
axes.fill_between(x,
                  MinGP99,
                  MaxGP99,
                  alpha=0.3,
                  facecolor='grey',
                  label='99% Range')
# Formatting
axes.set_xlim(0,41)
axes.set_ylim(0,20)
axes.set_xlabel('Profile')
axes.set_ylabel('Dune erosion (m$^3$/m)')
axes.grid()
axes.legend(framealpha=1)
# Calcualte and print r-squared
r_squared = pearsonr(obs,MeanGP)[0]**2
print('r2 =', format(r_squared,'.2f'))

## Figure 9 <a class="anchor" id="bullet-6"></a>
In the previous code and figures we have fixed the Cs parameter of the LEH04 model to be 1.5x10-3 and the number of ensembles to be 10,000. We will now expore the senstivity of the GP + LEH04 approach to variations in these two parameters. The code below is effectively the same as that used above except iterated over many Cs values. ***Note that this code block can take several hours to run.***

In [ ]:
# Define parameter space
Cs = np.logspace(-5,-1,50);
draws=10000
# Initialize result arrays
errorGP = np.zeros((len(june2011_profiles),len(Cs)))
errorST = np.zeros((len(june2011_profiles),len(Cs)))
errorGPmean = np.zeros((len(june2011_profiles),len(Cs),draws))
MaxGP = np.zeros((len(june2011_profiles),len(Cs),draws));
MinGP = np.zeros((len(june2011_profiles),len(Cs),draws));
MaxGP66 = np.zeros((len(june2011_profiles),len(Cs),draws));
MinGP66 = np.zeros((len(june2011_profiles),len(Cs),draws));
MaxGP90 = np.zeros((len(june2011_profiles),len(Cs),draws));
MinGP90 = np.zeros((len(june2011_profiles),len(Cs),draws));
MaxGP99 = np.zeros((len(june2011_profiles),len(Cs),draws));
MinGP99 = np.zeros((len(june2011_profiles),len(Cs),draws));
MeanGP = np.zeros((len(june2011_profiles),len(Cs),draws));
# loop through all the Cs indicies
for j in range(len(Cs)):
    # loop through the profile indicies
    for k in june2011_profiles:
        profile=june2011_profiles[k]
        #pull all relevant data out of the dictionary
        dv = profile['dv']
        zb = profile['zb']
        T = profile['Tp']
        GP_sample = np.squeeze(GP_samples[k,:,:]) # the GP runup draws for the profile
        # run the LEH04 model
        [SigDuneErosionGPD,zbmGPD] = LEH04ensembles(dv,zb,GP_sample,T,Cs[j])
        # record some results
        for n in range(draws):
            MeanGP[k,j,n] = np.mean(SigDuneErosionGPD[-1,0:n+1])
            MaxGP66[k,j,n] = np.percentile(SigDuneErosionGPD[-1,0:n+1],83)
            MinGP66[k,j,n] = np.percentile(SigDuneErosionGPD[-1,0:n+1],17)
            MaxGP90[k,j,n] = np.percentile(SigDuneErosionGPD[-1,0:n+1],95)
            MinGP90[k,j,n] = np.percentile(SigDuneErosionGPD[-1,0:n+1],5)
            MaxGP99[k,j,n] = np.percentile(SigDuneErosionGPD[-1,0:n+1],99.5)
            MinGP99[k,j,n] = np.percentile(SigDuneErosionGPD[-1,0:n+1],0.5)
    
        errorGPmean[k,j,:] = np.absolute(profile['dv_obs'] - MeanGP[k,j,:])
# record the capture statistics
PercentWithin66 = np.zeros((len(Cs),draws))
PercentWithin90 = np.zeros((len(Cs),draws))
PercentWithin99 = np.zeros((len(Cs),draws))
obs = np.zeros((len(june2011_profiles)))
for h in june2011_profiles:
    obs[h] = june2011_profiles[h]['dv_obs']  
for f in range(len(Cs)):
    for g in range(draws):
        within = np.where((MaxGP66[:,f,g]>=obs) & (MinGP66[:,f,g]<=obs))
        PercentWithin66[f,g] = 100*(len(within[0])/(len(june2011_profiles)));
        
        within = np.where((MaxGP90[:,f,g]>=obs) & (MinGP90[:,f,g]<=obs))
        PercentWithin90[f,g] = 100*(len(within[0])/(len(june2011_profiles)));
        
        within = np.where((MaxGP99[:,f,g]>=obs) & (MinGP99[:,f,g]<=obs))
        PercentWithin99[f,g] = 100*(len(within[0])/(len(june2011_profiles)));

In [ ]:
# Initialize the figure and axes
fig, axes = plt.subplots(nrows=4,ncols=1,figsize=(12/2.54,24/2.54))
plt.tight_layout(w_pad = 4, h_pad = 1)
# Plot MAE results
lineObjects2 = axes[0].semilogx(Cs,np.mean(errorGPmean[:,:,[4,9,19,99,999,9999]], axis=0))
axes[0].set_ylabel('MAE for all profiles (m$^3$/m)')
axes[0].set_ylim(0, 10) 
axes[0].set_xlim(10**-5,10**-1)
axes[0].grid(which='both')
axes[0].legend(lineObjects2,
               ('5','10','20','100','1000','10,000'),
               title='# Ensemble\n    Members',
               bbox_to_anchor=(0,0.5),
               loc=6,
               framealpha=1)
axes[0].text(7.2**-7, 10.4, 'A)', fontweight='bold', fontsize=12)
# Plot 99% capture results
axes[1].semilogx(Cs, PercentWithin99[:,[4,9,19,99,999,9999]])
axes[1].set_ylim(0, 100)
axes[1].set_xlim(10**-5,10**-1)
axes[1].set_ylabel('% of dune erosion \ndata within 99% range')
axes[1].set_xlim(10**-5,10**-1)
axes[1].grid(which='both') 
axes[1].text(7.2**-7, 104, 'B)', fontweight='bold', fontsize=12)
# Plot 95% capture results
axes[2].semilogx(Cs, PercentWithin90[:,[4,9,19,99,999,9999]])
axes[2].set_ylim(0, 100) 
axes[2].set_ylabel('% of dune erosion \ndata within 90% range')
axes[2].set_xlim(10**-5,10**-1)
axes[2].grid(which='both') 
axes[2].text(7.2**-7, 104, 'C)', fontweight='bold', fontsize=12)
# Plot 66% capture results
axes[3].semilogx(Cs, PercentWithin66[:,[4,9,19,99,999,9999]])
axes[3].set_xlabel('Cs')
axes[3].set_ylim(0, 100) 
axes[3].set_ylabel('% of dune erosion \ndata within 66% range')
axes[3].set_xlim(10**-5,10**-1)
axes[3].grid(which='both') 
axes[3].text(7.2**-7, 104, 'D)', fontweight='bold', fontsize=12)
# Print out some useful stats
Cs_5_ind = np.argmin(np.mean(errorGPmean[:,:,4],axis=0))
Cs_10_ind = np.argmin(np.mean(errorGPmean[:,:,9],axis=0))
Cs_20_ind = np.argmin(np.mean(errorGPmean[:,:,19],axis=0))
Cs_100_ind = np.argmin(np.mean(errorGPmean[:,:,99],axis=0))
Cs_1000_ind = np.argmin(np.mean(errorGPmean[:,:,999],axis=0))
Cs_10000_ind = np.argmin(np.mean(errorGPmean[:,:,9999],axis=0))
print('Ensemble Members | Optimum Cs')
print('               5 : ' + format(Cs[Cs_5_ind],'.4f'))
print('              10 : ' + format(Cs[Cs_10_ind],'.4f'))
print('              20 : ' + format(Cs[Cs_20_ind],'.4f'))
print('             100 : ' + format(Cs[Cs_100_ind],'.4f'))
print('            1000 : ' + format(Cs[Cs_1000_ind],'.4f'))
print('          10,000 : ' + format(Cs[Cs_10000_ind],'.4f'))
print('')
print('Ensemble Members | MAE')
print('               5 : ' + format(np.mean(errorGPmean[:,Cs_5_ind,4],axis=0),'.2f'))
print('              10 : ' + format(np.mean(errorGPmean[:,Cs_10_ind,9],axis=0),'.2f'))
print('              20 : ' + format(np.mean(errorGPmean[:,Cs_20_ind,19],axis=0),'.2f'))
print('             100 : ' + format(np.mean(errorGPmean[:,Cs_100_ind,99],axis=0),'.2f'))
print('            1000 : ' + format(np.mean(errorGPmean[:,Cs_1000_ind,999],axis=0),'.2f'))
print('          10,000 : ' + format(np.mean(errorGPmean[:,Cs_10000_ind,9999],axis=0),'.2f'))
print('')
print('Ensemble Members | 66% | 90% | 99%')
print('               5 : ' + 
      format(PercentWithin66[Cs_5_ind,4],'.0f') + '    ' +
      format(PercentWithin90[Cs_5_ind,4],'.0f') + '    ' +
      format(PercentWithin99[Cs_5_ind,4],'.0f'))
print('              10 : ' + 
      format(PercentWithin66[Cs_10_ind,9],'.0f') + '    ' +
      format(PercentWithin90[Cs_10_ind,9],'.0f') + '    ' +
      format(PercentWithin99[Cs_10_ind,9],'.0f'))
print('              20 : ' + 
      format(PercentWithin66[Cs_20_ind,19],'.0f') + '    ' +
      format(PercentWithin90[Cs_20_ind,19],'.0f') + '    ' +
      format(PercentWithin99[Cs_20_ind,19],'.0f'))
print('             100 : ' + 
      format(PercentWithin66[Cs_100_ind,99],'.0f') + '    ' +
      format(PercentWithin90[Cs_100_ind,99],'.0f') + '    ' +
      format(PercentWithin99[Cs_100_ind,99],'.0f'))
print('            1000 : ' + 
      format(PercentWithin66[Cs_1000_ind,999],'.0f') + '    ' +
      format(PercentWithin90[Cs_1000_ind,999],'.0f') + '    ' +
      format(PercentWithin99[Cs_1000_ind,999],'.0f'))
print('          10,000 : ' + 
      format(PercentWithin66[Cs_10000_ind,9999],'.0f') + '    ' +
      format(PercentWithin90[Cs_10000_ind,9999],'.0f') + '    ' +
      format(PercentWithin99[Cs_10000_ind,9999],'.0f'))